In [26]:
#Whole Numbers,2 Targets
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [27]:
walmart = pd.read_csv('walmartdata.csv')
walmart.head()

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68


In [28]:
totalave = walmart.groupby(["Store"])["Weekly_Sales"].mean()

In [29]:
for i in range(len(walmart)):
    walmart.loc[i, "Ave_Sales"] = totalave[walmart.loc[i, "Store"]]
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales,Ave_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90,1.555264e+06
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44,1.555264e+06
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17,1.555264e+06
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59,1.555264e+06
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68,1.555264e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6434,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95,7.859814e+05
6435,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07,7.859814e+05
6436,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36,7.859814e+05
6437,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53,7.859814e+05


In [30]:
walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Weekly_Sales","Ave_Sales"])

In [31]:
walmart.loc[walmart['Weekly_Sales'] > (1.15 * walmart["Ave_Sales"]), 'Label'] = 'Above' 
walmart.loc[walmart['Weekly_Sales'] < (1.15 * walmart["Ave_Sales"]), 'Label'] = 'Average'

In [32]:
walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Label"])

In [33]:
walmart["Label"].value_counts()

Average    5939
Above       500
Name: Label, dtype: int64

In [34]:
X = walmart.drop("Label", axis=1)
y = walmart["Label"]
print(X.shape, y.shape)

(6439, 4) (6439,)


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [36]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

# Scale and train x value datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [38]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model

LogisticRegression(random_state=42)

In [39]:
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(random_state=42)

In [40]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.9242079105404846
Testing Data Score: 0.9167701863354037


In [41]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
             'max_iter':[200, 500, 1000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [42]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.923, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.924, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] . C=1, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=newton-cg .............................
[CV] . C=1, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=newton-cg .............................
[CV] . C=1, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=newton-cg .............................
[CV] . C=1, max_iter=500, solver=newton-cg, score=0.923, total=   0.0s
[CV] C=1, max_iter=500, solver=newton-cg .............................
[CV] . C=1, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=liblinear .............................
[CV] . C=1, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=liblinear .............................
[CV] . C=1, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=1, max_iter=500, solver=liblinear .............................
[CV] .

[CV] . C=5, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=newton-cg .............................
[CV] . C=5, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=newton-cg .............................
[CV] . C=5, max_iter=500, solver=newton-cg, score=0.923, total=   0.0s
[CV] C=5, max_iter=500, solver=newton-cg .............................
[CV] . C=5, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=liblinear .............................
[CV] . C=5, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=liblinear .............................
[CV] . C=5, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=liblinear .............................
[CV] . C=5, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=5, max_iter=500, solver=liblinear .............................
[CV] .

[CV]  C=10, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=newton-cg ............................
[CV]  C=10, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=newton-cg ............................
[CV]  C=10, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=newton-cg ............................
[CV]  C=10, max_iter=500, solver=newton-cg, score=0.923, total=   0.0s
[CV] C=10, max_iter=500, solver=newton-cg ............................
[CV]  C=10, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=liblinear ............................
[CV]  C=10, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=liblinear ............................
[CV]  C=10, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=10, max_iter=500, solver=liblinear ............................
[CV]  

[CV] ..... C=50, max_iter=200, solver=saga, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=newton-cg ............................
[CV]  C=50, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=newton-cg ............................
[CV]  C=50, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=newton-cg ............................
[CV]  C=50, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=newton-cg ............................
[CV]  C=50, max_iter=500, solver=newton-cg, score=0.923, total=   0.0s
[CV] C=50, max_iter=500, solver=newton-cg ............................
[CV]  C=50, max_iter=500, solver=newton-cg, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=liblinear ............................
[CV]  C=50, max_iter=500, solver=liblinear, score=0.924, total=   0.0s
[CV] C=50, max_iter=500, solver=liblinear ............................
[CV]  

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:    2.5s finished


GridSearchCV(estimator=LogisticRegression(random_state=42),
             param_grid={'C': [1, 5, 10, 50], 'max_iter': [200, 500, 1000],
                         'solver': ['newton-cg', 'liblinear', 'sag', 'saga']},
             verbose=3)

In [43]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'max_iter': 200, 'solver': 'newton-cg'}
0.9242079404413264


In [44]:
logreg = LogisticRegression(C=1, max_iter=200, solver= 'newton-cg', random_state=42)
logreg.fit(X_train_scaled, encoded_y_train)
print(f"Training Data Score: {logreg.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {logreg.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.9242079105404846
Testing Data Score: 0.9167701863354037


In [45]:
target = walmart["Label"]
target_names = ["Above", "Average"]

In [46]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf = rf.fit(X_train_scaled, encoded_y_train)
rf.score(X_test_scaled, encoded_y_test)

0.9397515527950311

In [19]:
feature_names = walmart.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.3311428636926653, 'CPI'),
 (0.2694866326503375, 'Fuel_Price'),
 (0.20966494957819504, 'Temperature_C'),
 (0.18970555407880219, 'Unemployment')]

In [385]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [386]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [387]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [47]:
ranfor = RandomForestClassifier(n_estimators=500, random_state=42)
ranfor = ranfor.fit(X_train_scaled, encoded_y_train)
ranfor.score(X_test_scaled, encoded_y_test)

0.9397515527950311

In [48]:
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train_scaled, encoded_y_train)
clf.score(X_test_scaled, encoded_y_test)

0.9254658385093167

In [436]:
feature_names = walmart.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.36486851439318435, 'CPI'),
 (0.27004417922651425, 'Fuel_Price'),
 (0.182869784976228, 'Unemployment'),
 (0.1822175214040735, 'Temperature_C')]

In [391]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(clf, param_grid, verbose=3)

In [392]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [393]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [49]:
dectre = tree.DecisionTreeClassifier(random_state=42)
dectre = dectre.fit(X_train_scaled, encoded_y_train)
dectre.score(X_test_scaled, encoded_y_test)

0.9254658385093167

In [50]:
dectre_predictions = dectre.predict(X_test_scaled)

In [51]:
ranfor_predictions = ranfor.predict(X_test_scaled)

In [52]:
print(classification_report(encoded_y_test, ranfor_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.72      0.45      0.55       134
     Average       0.95      0.98      0.97      1476

    accuracy                           0.94      1610
   macro avg       0.84      0.72      0.76      1610
weighted avg       0.93      0.94      0.93      1610



In [53]:
print(classification_report(encoded_y_test, dectre_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.55      0.57      0.56       134
     Average       0.96      0.96      0.96      1476

    accuracy                           0.93      1610
   macro avg       0.76      0.76      0.76      1610
weighted avg       0.93      0.93      0.93      1610



In [82]:
predix = pd.read_csv('prophet_predictions.csv')
predix = predix.drop(['Unnamed: 0'], axis=1)
predix.head()

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06


In [83]:
predix
X = predix.drop("Date", axis=1)
X = X.drop("Store", axis=1)
X = X.drop("Weekly_Sales", axis=1)
X

,Fuel_Price,CPI,Unemployment,Temperature_C
0,3.487440,223.461145,6.627970,17.659895
1,3.472333,223.443172,6.634452,16.448992
2,3.456623,223.409742,6.623966,15.104578
3,3.441573,223.372609,6.592424,13.620766
4,3.423539,223.358492,6.556789,12.028795
...,...,...,...,...
2470,4.393188,195.136141,8.878137,16.570370
2471,4.400800,195.177228,8.872678,15.371153
2472,4.394871,195.161653,8.877326,13.409695
2473,4.380892,195.115184,8.887845,11.328064


In [84]:
predix_scaled = X_scaler.transform(X)

In [85]:
ranfor_labels = ranfor.predict(predix_scaled)

In [86]:
ranfor_labels = label_encoder.inverse_transform(ranfor_labels)
ranfor_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [87]:
ranfor_predix = predix
ranfor_predix["Label"] = ranfor_labels
ranfor_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,Average
...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,Average


In [88]:
ranfor_predix["Label"].value_counts()

Average    2461
Above        14
Name: Label, dtype: int64

In [73]:
logreg_labels = logreg.predict(predix_scaled)

In [74]:
logreg_labels = label_encoder.inverse_transform(logreg_labels)
logreg_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [76]:
logreg_predix = predix
logreg_predix["Label"] = logreg_labels
logreg_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,Average
...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,Average


In [77]:
logreg_predix["Label"].value_counts()

Average    2475
Name: Label, dtype: int64

In [63]:
dectre_labels = dectre.predict(predix_scaled)

In [64]:
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [65]:
dectre_predix = predix
dectre_predix["Label"] = dectre_labels
dectre_predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,Average
...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,Average


In [66]:
dectre_predix["Label"].value_counts()

Average    2215
Above       260
Name: Label, dtype: int64

In [67]:
predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Label
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,Average
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,Average
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,Average
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,Average
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,Average
...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,Average
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,Average
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,Average
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,Average


In [69]:
predix = pd.read_csv('prophet_predictions.csv')

In [70]:
for i in range(len(predix)):
    predix.loc[i, "Ave_Sales"] = totalave[predix.loc[i, "Store"]]
predix

,Unnamed: 0,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales
0,0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,1.555264e+06
1,1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,1.555264e+06
2,2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,1.555264e+06
3,3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,1.555264e+06
4,4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,1.555264e+06
...,...,...,...,...,...,...,...,...,...
2470,2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,7.859814e+05
2471,2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,7.859814e+05
2472,2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,7.859814e+05
2473,2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,7.859814e+05


In [71]:
predix.loc[predix['Weekly_Sales'] > (1.15 * predix["Ave_Sales"]), 'Label'] = 'Above' 
predix.loc[predix['Weekly_Sales'] < (1.15 * predix["Ave_Sales"]), 'Label'] = 'Average'
proph_predix = predix

In [72]:
proph_predix["Label"].value_counts()

Average    2071
Above       404
Name: Label, dtype: int64

In [89]:
ranfor_predix["Label"].value_counts()

Average    2461
Above        14
Name: Label, dtype: int64

In [79]:
logreg_predix["Label"].value_counts()

Average    2475
Name: Label, dtype: int64

In [80]:
dectre_predix["Label"].value_counts()

Average    2215
Above       260
Name: Label, dtype: int64